# Test Bench

Test bench is a class we created that takes in some perediction model and creates multiple instances of that model. Each instance is trained and tested on different data, it is used to easily access the quality of each model on multiple datasets.

## How Is It Used?

First define a class that has these functions:
1. `learn_from_data_set`
2. `predict`

In [1]:
import numpy as np
import src.test_bench as bench

In [2]:
class DumbPredictor:
    def __init__(self):
        print("Constructor called.")
        self.print_once = True

    def learn_from_data_set(self, training_data_set):
        print("Training started.")
        print("What does a dataframe to learn on look like?")
        display(training_data_set[0])
        print("Training ending.")

    def predict(self, ts_as_df_start, how_much_to_predict):
        if self.print_once:
            self.print_once = False
            print("What does a dataframe to predict look like?")
            display(ts_as_df_start)
        ts_as_np = ts_as_df_start["sample"].to_numpy()
        res = np.resize(ts_as_np, how_much_to_predict)
        # these checks will also be done by the testbench
        assert isinstance(res, np.ndarray)
        assert len(res) == how_much_to_predict
        assert res.shape == (how_much_to_predict,)
        assert res.dtype == np.float64
        return res

After the model has been defined (including how you choose to you use each sample for learning, what optimization algorithms and so on) you can send the model to the test_bench to be used for prediction on multiple models.
Notice that the class you denfine need to have these methods with the same argument names.

Let's now run the test bench on this class.

In [ ]:
tb = bench.TestBench(
    class_to_test=DumbPredictor,
    metrics_and_apps_to_test=[
        ("container_mem", "kube-rbac-proxy"),
        ("container_cpu", "collector"),
        ("node_mem", "moc/smaug"),
    ],
    path_to_data="./data/"
)
tb.run_training_and_tests()

[TEST BENCH] Powering on test bench
[TEST BENCH] testing metric='container_mem', app='container_mem'.
[TEST BENCH] Fetching data for metric='container_mem', app='container_mem'.
